In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp
import pygeos
import rtree
from time import sleep


ModuleNotFoundError: No module named 'pygeos'

In [ ]:
gp.read_file('../raw_data/housing_data.shp')

# Loading migration data

In [ ]:
mig_data = pd.read_csv('../raw_data/EWRMIGRA202012E_Matrix.csv', sep=';')
gen_data = pd.read_csv('../raw_data/EWR202012E_Matrix.csv', sep=';')

migration = mig_data[['ZEIT','RAUMID', 'MH_E', 'MH_EM', 'MH_EW']].merge(gen_data[['RAUMID', 'E_E' ,'E_EM', 'E_EW', 'E_E00_01',
       'E_E01_02', 'E_E02_03', 'E_E03_05', 'E_E05_06', 'E_E06_07', 'E_E07_08',
       'E_E08_10', 'E_E10_12', 'E_E12_14', 'E_E14_15', 'E_E15_18', 'E_E18_21',
       'E_E21_25', 'E_E25_27', 'E_E27_30', 'E_E30_35', 'E_E35_40', 'E_E40_45',
       'E_E45_50', 'E_E50_55', 'E_E55_60', 'E_E60_63', 'E_E63_65', 'E_E65_67',
       'E_E67_70', 'E_E70_75', 'E_E75_80', 'E_E80_85', 'E_E85_90', 'E_E90_95',
       'E_E95_110']], on='RAUMID')

migration.to_csv('../data/migration_data.csv')

# Loading school data

In [ ]:
schools_list = pd.read_excel('../raw_data/Schulen.xlsx')

schools_list = schools_list.drop(index=0)
new_header = schools_list.iloc[0] #grab the first row for the header
schools_list = schools_list[1:] #take the data less the header row
schools_list.columns = new_header #set the header row as the df header

schools_list = schools_list[schools_list.Traeger!='privat']
schools_list = schools_list[['BSN', 'NAME', 'Bezirk', 'PLZ', 'Adresse']]

schools_list

In [ ]:
schools_stat = pd.read_excel('../raw_data/schools_stat.xlsx')

new_header = schools_stat.iloc[0] #grab the first row for the header
schools_stat = schools_stat[1:] #take the data less the header row
schools_stat.columns = new_header #set the header row as the df header

schools_stat.columns = ['School_year', 'BSN', 'NAME', 'children', 'children_f',
       'children_m', 'teachers', 'teachers_f', 'teachers_m']

schools_stat = schools_stat.drop(columns='NAME')

In [ ]:
schools = schools_list.merge(schools_stat, how='left', on='BSN')
schools

In [ ]:
sum_children = schools.children.sum()
schools["teachers_rate"]= schools.teachers / schools.children
schools["capacity_rate"]= schools.children / sum_children

schools.to_csv('../data/schools.csv', index=False)

pd.read_csv('../data/schools.csv')

## Fixing formatting issues 

In [ ]:
schools['Adresses'] = schools.Adresse + ' ' + schools.PLZ.astype(str) + ' Berlin'

schools['Adresses'].iloc[111] = 'Spandauer Damm 205 14050 Berlin'
schools['Adresses'].iloc[132] = 'Maikaerpfad 30 14055 Berlin'

schools['Adresses'].iloc[132] = 'Maikaerpfad 30 14055 Berlin'

schools['Adresses'].iloc[153] = 'Raenitzer Steig 12 13593 Berlin'

schools['NAME'].iloc[171] = 'Flaeming-Grundschule'
schools['Adresses'].iloc[171] = 'Illstr. 4 12161 Berlin'
schools['Adresses'].iloc[250]= 'An den Achterhöfen 13 12349 Berlin'

schools['Adresses'].iloc[255]= 'Wildhüterweg 5, 12353 Berlin'

schools['Adresses'].iloc[171] = 'Ludwigsfelder Str. 43 14165 Berlin'

schools['Adresses'].iloc[258] = 'Haenselstr. 6 12057 Berlin'
schools['NAME'].iloc[258] = 'Schule in der Koellnischen Heide'

schools['Adresses'].iloc[262] = 'Haenselstr. 14 12437 Berlin'

schools['Adresses'].iloc[307] = 'Erich-Kaestner-Straße 64 12619 Berlin'

schools['Adresses'].iloc[317] = 'Bernhard-Baestlein-Straße 22 10367 Berlin'

schools['Adresses'].iloc[340] = 'Dolgenseestraße 60 10319 Berlin'

schools['Adresses'].iloc[341] = 'Bernhard-Baestlein-Straße 56 10367 Berlin'

schools['Adresses'].iloc[350] = 'Hollaenderstraße 25-30, 13407 Berlin'
schools['NAME'].iloc[350] = 'Grundschule am Schaersee'

schools['Adresses'].iloc[356] = 'Tornower Weg 26 13439 Berlin'


In [ ]:
schools

## Geocoding schools addresses

In [ ]:
schools_location = gp.GeoDataFrame({'location':range(0,379)})


In [ ]:
for i in schools.index: 
    sleep(5)
    schools_location['location'].iloc[i] = gp.tools.geocode(schools.Adresses[i])['geometry'].iloc[0]

In [ ]:
schools_loc1 = gp.tools.geocode(schools.Adresses[0:100])

In [ ]:
schools_loc3 = gp.tools.geocode(schools.Adresses[100:200])

In [ ]:
schools_loc3 = gp.tools.geocode(schools.Adresses[200:300])

In [ ]:
schools_loc4 = gp.tools.geocode(schools.Adresses[300:])

In [ ]:
schools_location = pd.concat([schools_loc1, schools_loc2, schools_loc3, schools_loc4], axis=0)

In [ ]:
schools['location'] = schools_location['geometry']

In [ ]:
indexes = [250, 255, 258,  260, 262]

for i in indexes: 
    schools[['location']].iloc[i] = gp.tools.geocode(schools.Adresses[i])['geometry'].iloc[0]
    schools['Adresses'].iloc[i] = schools['Adresses'].iloc[i]

In [ ]:
indexes = [307, 317, 340, 341, 350, 356]

for i in indexes: 
    schools[['location']].iloc[i] = gp.tools.geocode(schools.Adresses[i])['geometry'].iloc[0]
    schools['Adresses'].iloc[i] = schools['Adresses'].iloc[i]

In [ ]:
schools[schools.location.isna()]

In [ ]:
schools_geo = gp.GeoDataFrame(
    schools, geometry=schools.location)

In [ ]:
schools_geo.isna().sum()

In [ ]:
schools_geo = schools_geo.to_crs(epsg=(3857))

In [ ]:
school_districts = gp.read_file('/Users/Safia/Documents/Mapping_Social_Inequalities/Maps/ESB2012_WGS84_EWR2012-12/ESB2012_WGS84_EWR2012-12.shp')


In [ ]:
schools_points = school_districts.sjoin(schools_geo, how='right', predicate = 'intersects')

In [ ]:
#schools['location'].iloc[204] = gp.tools.geocode(schools.Adresses[204])['geometry'].iloc[0]

In [ ]:
schools_points[schools_points.ESB.isna() == True]

In [ ]:
missings = schools_geo[schools_points.ESB.isna() == True]

In [ ]:
missings_nearest_merge = school_districts.sjoin_nearest(
        missings, how='right')

In [ ]:
missings_nearest_merge

In [ ]:
schools_points.iloc[255] = missings_nearest_merge.iloc[0]
schools_points.iloc[260] = missings_nearest_merge.iloc[1]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
school_districts.plot(column=('ESB'), ax=ax)
schools_points.plot(ax = ax)


In [ ]:
ESB2018 = gp.read_file('/Users/Safia/Documents/Mapping_Social_Inequalities/Maps/Berlin_Grundschuleinzugsbereiche20182019_25833.shp')




In [ ]:
ESB2018.head()

In [ ]:
ESB2018.plot(column='ESB', figsize=(15,15))

In [ ]:
school_districts.plot(column='ESB', figsize=(15,15))

In [ ]:
schools_points.to_csv('schools_location')

In [ ]:
schools_points.head()

In [ ]:
schools_info = schools_points[['BSN', 'NAME', 'ESB', 'BEZ', 'FLAECHE_HA','School_year', 'children',
       'children_f', 'children_m', 'teachers', 'teachers_f', 'teachers_m',
       'teachers_rate', 'capacity_rate', 'Adresses','Bezirk', 'location', 'geometry','EW_insgesa', 'EW_U6',
       'EW_6U14', 'EW_14U18', 'EW_18U25', 'EW_25U45', 'EW_45U65', 'EW_65plus',
       ]]

In [ ]:
schools_info

In [ ]:
shape2 = gp.read_file('/Users/Safia/Documents/Mapping_Social_Inequalities/Maps/blockflaechen22.shp')

In [ ]:
shape2 = shape2.to_crs(epsg=3857)

In [ ]:
shape2.head(3)

In [ ]:
shape.head(3)

In [ ]:
shape3 = shape.overlay(shape2)

In [ ]:
shape3.head(3)

In [ ]:
shape3.plot(figsize=[15,15])